In [2]:
'''importing required lib'''
import numpy as np # to deal with numbers and array
import pandas as pd #for handling dataframes
from sklearn.utils import shuffle #use to shuffle dataframe

''' lib required for creating a model  i.e text data into array of matrix'''
from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model
# for converting text data into array 

''' lib required for cleaning of data'''
from nltk.corpus import stopwords #containing stopwords
from nltk.stem.porter import PorterStemmer # use to fing a core meaning of words 
from nltk.stem.wordnet import WordNetLemmatizer
import re #used for regular expression operation
import os
import nltk
import pickle


In [3]:
stop_words = set(stopwords.words('english'))
print(stop_words)
new_words = ["RT",'rt'] #creating a list of extra stop words
stop = stop_words.union(new_words) #adding words into corpus

{'s', 'whom', 'your', "you've", 'most', "won't", 'were', 'having', 'with', "doesn't", 'there', 'do', 'yourself', 'him', 'it', 'll', 'against', 'more', "that'll", 'and', 'that', 'after', 'who', 'hadn', 'had', 'why', 'you', 'about', 'than', 't', 'too', 'between', "you're", 'does', 'again', 'are', 'wasn', 'y', 'am', 'shouldn', 'nor', "should've", 'by', 'for', 'from', 'been', 'themselves', 'they', 'being', 'them', 'shan', 're', "mustn't", 'myself', 'hers', 'or', 'above', 'up', "you'd", 'where', 'haven', "don't", 'each', 'until', 'wouldn', 'we', 'the', 'not', 'own', 'through', 'other', 'their', 'off', "isn't", 'few', 'have', 'how', 'further', 'just', 'our', 'd', 'this', 'her', 'if', 'needn', 'so', 'on', 'when', 'will', "shouldn't", 'both', 'ain', "wasn't", 'aren', 'a', "hasn't", 'during', 'same', 'won', "mightn't", "hadn't", "it's", 'o', 'once', 'ma', 'mustn', 'can', "couldn't", 'theirs', 'no', 'couldn', 'here', "didn't", "shan't", 'into', 'below', "she's", 'these', 'what', 'has', 'under', 

In [4]:
'''
Input : The whole dataframe will be send to the function
Output : Column with clean tweets tweet will be added
Function perform following operation:
       1. Clean text from html tags
       2. Clean text from punctuations and special characters
       3. Retain only non-numeric Latin characters with lenght > 2
       4. Remove stopwords from the sentence
       5. Apply stemming to all the words in the sentence
'''
def clean_tweets(dataset):
    corpus = []
    for i in range(0, dataset.shape[0]):
        text=dataset['tweet_text'][i]
        #hte line data.shape will return [row size,columnsize ] from which we are taking row size using index 0
        
        #remove @mentions
        rg=re.compile('@[^, ]*\s*')
        text=re.sub(rg," ",text)

        #remove urls
        rg=re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text=re.sub(rg," ",text)

        #Remove punctuations
        text = re.sub('[^a-zA-Z0-9]', ' ',text)

        #Convert to lowercase
        text = text.lower()

        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

        # remove special characters and digits
        #text=re.sub("(\\d|\\W)+"," ",text)
        text=re.sub("(\\W)+"," ",text)

        ##Convert to list from string
        text = text.split()

        ##Stemming
        ps=PorterStemmer()    #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in
                stop_words and len(word)>2]
        text = " ".join(text)
        corpus.append(text)
    dataset['preprocessed']=corpus
    return dataset

In [5]:
from sklearn.svm import LinearSVC
from sklearn import metrics
pickle_model=open("linearSvc.pickle" ,'rb')
linear_svc_model=pickle.load(pickle_model)

In [6]:
pickle_tfidf =open("tfidf_model_11452.pickle" ,'rb')
tf_idf_vect=pickle.load(pickle_tfidf)

In [7]:
dataframe=pd.read_csv('other_useful_information.csv',encoding = 'unicode_escape')
dataframe=dataframe.drop(['Unnamed: 0'],axis=1)
dataframe.head()

,label,tweet_id,tweet_text
0,other_useful_information,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...
1,other_useful_information,'592590231519555584',NepalÃÂs Slowing Economy Set for Freefall Wi...
2,other_useful_information,'593470045814181888',Food crisis imminent for 1.4 million ppl in Ka...
3,other_useful_information,'593155728128516097',Canadian phone companies waive fees for calls ...
4,other_useful_information,'593609036144214016',RT @XuscaSoleNY: https://t.co/o1RqfCCmKm don't...


In [8]:
dataframe.shape

(6231, 3)

In [9]:
dataframe=clean_tweets(dataframe)# function calling for preprocessed tweet
dataframe.head()

,label,tweet_id,tweet_text,preprocessed
0,other_useful_information,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...,reached kathmandu finally lot indian stranded ...
1,other_useful_information,'592590231519555584',NepalÃÂs Slowing Economy Set for Freefall Wi...,nepal slowing economy set freefall without wor...
2,other_useful_information,'593470045814181888',Food crisis imminent for 1.4 million ppl in Ka...,food crisis imminent million ppl kathmandu lon...
3,other_useful_information,'593155728128516097',Canadian phone companies waive fees for calls ...,canadian phone company waive fee call nepal gl...
4,other_useful_information,'593609036144214016',RT @XuscaSoleNY: https://t.co/o1RqfCCmKm don't...,believe earthquake happen naturally mother nat...


In [10]:
dataframe.shape

(6231, 4)

In [18]:
processed_data=dataframe['preprocessed'].values
print(processed_data[1245:1255])
print(len(processed_data))

['typhoon hagupit wreaks havoc leadin filipino environmental voice silencd clim8 talk climatechange'
 'batangas province storm signal high alert vice gov tell rubyph'
 'updated 1pm today psws rubyph added area signal negro oriental occidental amp guimaras'
 'typhoon hagupit bear philippine bbc news'
 'cancelled flight due rubyph naia terminal saturday december 2014 via'
 'click open street map helped limit devastation typhoon hagupit'
 'sky destiny cablelink cignal rubyph'
 'advisory class manila city suspended tomorrow december 2014 level walangpasok sabi woooo'
 'daylight brings terrifying view super typhoon aiming philippine'
 'please advised class lpu manila suspended tom dec 2014 stay safe dry everyone rubyph walangp']
6231


In [12]:
vectorized_data=tf_idf_vect.transform(processed_data)

In [13]:
vectorized_data


<6231x11452 sparse matrix of type '<class 'numpy.float64'>'
	with 49179 stored elements in Compressed Sparse Row format>

In [19]:
#y_pred=linear_svc_model.predict(vectorized_data)
print(y_pred[1245:1255])
print(len(y_pred))

['infrastructure_and_utilities_damage' 'prevention_or_caution_and_advice'
 'prevention_or_caution_and_advice' 'prevention_or_caution_and_advice'
 'prevention_or_caution_and_advice' 'infrastructure_and_utilities_damage'
 'prevention_or_caution_and_advice' 'prevention_or_caution_and_advice'
 'prevention_or_caution_and_advice' 'sympathy_and_emotional_support']
6231


In [15]:
dataframe['label']=y_pred
dataframe.head()

,label,tweet_id,tweet_text,preprocessed
0,infrastructure_and_utilities_damage,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...,reached kathmandu finally lot indian stranded ...
1,donation_needs_or_offers_or_volunteering_services,'592590231519555584',NepalÃÂs Slowing Economy Set for Freefall Wi...,nepal slowing economy set freefall without wor...
2,donation_needs_or_offers_or_volunteering_services,'593470045814181888',Food crisis imminent for 1.4 million ppl in Ka...,food crisis imminent million ppl kathmandu lon...
3,donation_needs_or_offers_or_volunteering_services,'593155728128516097',Canadian phone companies waive fees for calls ...,canadian phone company waive fee call nepal gl...
4,sympathy_and_emotional_support,'593609036144214016',RT @XuscaSoleNY: https://t.co/o1RqfCCmKm don't...,believe earthquake happen naturally mother nat...


In [20]:
dataframe['label'].value_counts()

prevention_or_caution_and_advice                            2716
infrastructure_and_utilities_damage                          991
donation_needs_or_offers_or_volunteering_services            771
affectedPeople_or_injured_or_dead_people_or_DeathReports     698
sympathy_and_emotional_support                               494
disease_transmission_or_disease_signs_or_symptoms            288
missing_trapped_or_found_people                              154
treatment                                                    119
Name: label, dtype: int64

In [21]:
dataframe.shape

(6231, 4)

In [22]:
with open("dataset6231.pickle" ,'wb') as f:
    pickle.dump(dataframe,f)

###  WE HAVE CATEGORISED OTHER USEFUL INFORMATION INTO OUR DIFFERENT CATEGORY 

In [ ]:
'''
In next ipython notebook we are going to concat this other useful information dataset of 6231 into main dataset

cocatination of all dataset will be done "concatination of all dataset.iypnb "

'''